In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
import time
import math
import os
import sys
import matplotlib.pyplot as plt

In [3]:
# download file to tmp/data.txt
!wget -O tmp/data.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-25 22:28:16--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: 'tmp/data.txt'

     0K .......... .......... .......... .......... ..........  4% 4.56M 0s
    50K .......... .......... .......... .......... ..........  9% 7.48M 0s
   100K .......... .......... .......... .......... .......... 13% 19.4M 0s
   150K .......... .......... .......... .......... .......... 18% 8.90M 0s
   200K .......... .......... .......... .......... .......... 22% 25.7M 0s
   250K .......... .......... .......... .......... .......... 27% 12.2M 0s
   300K .......... .......... .......... .......... .......... 32% 23.9M 0s
  

In [4]:
with open('tmp/data.txt', 'r') as f:
    text = f.read()
print('text length:', len(text))

text length: 1115394


In [5]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print('vocab size:', vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65


In [7]:
# Create mapping from character to index and vice versa
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# store in tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100]) # the 100 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

# print('----')

# for b in range(batch_size): # batch dimension
#     for t in range(block_size): # time dimension
#         context = xb[b, :t+1]
#         target = yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [13]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, x, targets=None):
        if targets is None:
            # we're doing inference, so we don't have targets
            # in this case, x is a batch of contexts
            # we need to return a batch of predictions

            # if x is None, we're generating from scratch
            if x is None:
                x = batch_size

            # generate x batches
            if isinstance(x, int):
                x = torch.zeros(x, 1, dtype=torch.long)
            return self.table(x), None
        else:
            # we're training, so we do have targets
            logits = self.table(x)
            B, T, C = logits.shape  # batch, time, channel
            logits_flat = logits.view(B * T, C)
            loss = F.cross_entropy(logits_flat, targets.view(B * T))

            return logits_flat, loss

    def generate(self, x, max_len_new):
        for _ in range(max_len_new):
            logits, loss = self(x)
            probs = F.softmax(logits[:, -1], dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            x = torch.cat([x, next_token], dim=1)

        return x

In [14]:
model = BigramLanguageModel(vocab_size)
print(model)
logits, loss = model(xb, yb)
print('logits:', logits.shape)
print('loss:', loss)

print(decode(model.generate(xb, 10)[0].tolist()))

BigramLanguageModel(
  (table): Embedding(65, 65)
)
logits: torch.Size([32, 65])
loss: tensor(5.0364, grad_fn=<NllLossBackward>)
Let's heXJamq!iui$


In [15]:
# train the model
optimizer = optim.AdamW(model.parameters(), lr=0.004)
batch_size = 32
for step in range(10000):
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 500 == 0:
        print(f'step: {step}, loss: {loss.item()}')

print(f'step: {step}, loss: {loss.item()}')

step: 0, loss: 4.532054424285889


step: 500, loss: 3.228471517562866
step: 1000, loss: 2.726747751235962
step: 1500, loss: 2.483638048171997
step: 2000, loss: 2.423741340637207
step: 2500, loss: 2.434169054031372
step: 3000, loss: 2.4106667041778564
step: 3500, loss: 2.3632164001464844
step: 4000, loss: 2.420254945755005
step: 4500, loss: 2.3026375770568848
step: 5000, loss: 2.629218816757202
step: 5500, loss: 2.5279862880706787
step: 6000, loss: 2.4539856910705566
step: 6500, loss: 2.515232563018799
step: 7000, loss: 2.5164709091186523
step: 7500, loss: 2.540008068084717
step: 8000, loss: 2.460096836090088
step: 8500, loss: 2.530355930328369
step: 9000, loss: 2.407350540161133
step: 9500, loss: 2.621864080429077
step: 9999, loss: 2.3662900924682617


In [16]:
print(decode(model.generate(torch.tensor([encode("LUCENT")]), 100)[0].tolist()))

LUCENTER: und howiste ty dyotrd,
Theal lerno, y va f m my mulde ben s, r bet!
AMAs sod ke alved.
Thup sthe
